In [1]:
# !git clone https://github.com/awslabs/rag-qa-arena
# !mkdir rag-qa-benchmarking
# !mv rag-qa-arena/data/annotations_science_with_citation.jsonl ./rag-qa-benchmarking/
# !rm -rf rag-qa-arena

Cloning into 'rag-qa-arena'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 55 (delta 15), reused 41 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 3.59 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (15/15), done.
mkdir: rag-qa-benchmarking: File exists


In [2]:
# !curl https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz --output lotte.tar.gz
# !tar -xvzf lotte.tar.gz
# ! cp lotte/science/test/collection.tsv ./rag-qa-benchmarking/science_test_collection.tsv
# ! rm lotte.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3410M  100 3410M    0     0  5161k      0  0:11:16  0:11:16 --:--:-- 5196k2  0:10:15 5207k02  0:01:16  0:09:46 5216k2  0:01:23  0:09:39 5207k   0  5258k      0  0:11:04  0:01:49  0:09:15 5229k5235k     0  0:11:05  0:02:19  0:08:46 5220k:05  0:02:31  0:08:34 5229k5  0:03:03  0:08:02 5206k11:06  0:03:26  0:07:40 5165k1:08  0:04:16  0:06:52 5195k  0:04:20  0:06:48 5181k:35  0:06:33 5188k 0  0:11:09  0:04:48  0:06:21 5181k  5217k      0  0:11:09  0:04:56  0:06:13 5113k      0  0:11:09  0:05:09  0:06:00 5124k0     0  5211k      0  0:11:10  0:05:15  0:05:55 5121k    0  5202k      0  0:11:11  0:05:52  0:05:19 5122k    0  0:11:11  0:05:57  0:05:14 5120k 0  0:11:12  0:06:16  0:04:56 5120k0  0:11:12  0:06:50  0:04:22 5116k89k      0  0:11:12  0:06:57  0:04:15 5117k  5180k      0  0:11:14  0:08:01  0:03:13 5122k0  5179k      0  0:11:14  

In [3]:
import os

import pandas as pd

In [4]:
questions = pd.read_json("rag-qa-benchmarking/annotations_science_with_citation.jsonl", lines=True)

docs = pd.read_csv(
    "rag-qa-benchmarking/science_test_collection.tsv",
    sep="\t",
    header=None,
)
docs.columns = ["doc_id", "doc_text"]

In [17]:
percentage_to_use = 1
proportion_to_use = percentage_to_use / 100
papers_directory = "rag-qa-benchmarking/lfrqa"
amount_of_docs_to_use = int(len(docs) * proportion_to_use)
partial_docs = docs.head(amount_of_docs_to_use)
print(f"Using {amount_of_docs_to_use} out of {len(docs)} documents")

Using 16940 out of 1694041 documents


In [18]:
! find rag-qa-benchmarking/lfrqa/science_docs_for_paperqa/files -type f -delete
! rm -rf rag-qa-benchmarking/lfrqa/science_docs_for_paperqa_index

In [19]:
os.makedirs(f"{papers_directory}/science_docs_for_paperqa/files", exist_ok=True)
for i, row in partial_docs.iterrows():
    doc_id = row["doc_id"]
    doc_text = row["doc_text"]

    with open(f"{papers_directory}/science_docs_for_paperqa/files/{doc_id}.txt", "w", encoding="utf-8") as f:
        f.write(doc_text)

    if i % int(len(partial_docs) * 0.05) == 0:
        progress = (i + 1) / len(partial_docs)
        print(f"Progress: {progress:.2%}")


Progress: 0.01%
Progress: 5.01%
Progress: 10.01%
Progress: 15.01%
Progress: 20.01%
Progress: 25.01%
Progress: 30.01%
Progress: 35.01%
Progress: 40.01%
Progress: 45.01%
Progress: 50.01%
Progress: 55.01%
Progress: 60.01%
Progress: 65.01%
Progress: 70.01%
Progress: 75.01%
Progress: 80.01%
Progress: 85.01%
Progress: 90.01%
Progress: 95.01%


In [20]:
manifest = partial_docs.copy()
manifest["file_location"] = manifest["doc_id"].apply(lambda x: f"files/{x}.txt")
manifest["doi"] = ""
manifest["overwrite_fields_from_metadata"] = False
manifest["title"] = manifest["doc_id"].apply(lambda x: x)
manifest["key"] = manifest["title"]
manifest["docname"] = manifest["title"]
manifest["citation"] = "_"
manifest = manifest.drop(columns=["doc_id", "doc_text"])
manifest.to_csv(f"{papers_directory}/science_docs_for_paperqa/manifest.csv", index=False)

In [21]:
partial_questions = questions[
    questions.gold_doc_ids.apply(lambda ids: all(id < amount_of_docs_to_use for id in ids))
]
partial_questions.to_csv(f"{papers_directory}/questions.csv", index=False)

In [22]:
len(partial_questions)

62